In [1]:
import json
import os
import time
from typing import Any

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader
from torchvision import transforms
import pandas as pd

from sklearn.metrics import f1_score   
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import warnings
warnings.filterwarnings('always')  # "error", "ignore", "always", "default", "module" or "once"

In [2]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [3]:
from  RoadImageReader import RoadImageReader

In [4]:
from GaborNN import GaborNN

In [5]:
transform = transforms.Compose(
        [
            transforms.ToPILImage(),
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ]
    )

In [6]:
train_set = RoadImageReader(
        root_dir=os.path.join("C:\\Users\\91701\\OneDrive\\thesiscode\\data", "train"), transform=transform
    )
test_set = RoadImageReader(
        root_dir=os.path.join("C:\\Users\\91701\\OneDrive\\thesiscode\\data", "test"), transform=transform
    )

In [7]:
train = DataLoader(train_set, batch_size=128, shuffle=True, num_workers=7)
test = DataLoader(test_set, batch_size=128, shuffle=True, num_workers=7)

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [9]:
net = GaborNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters())
#scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.01, max_lr=0.1,cycle_momentum=False)

C:\ProgramData\Anaconda3\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [10]:
epochs=30

In [11]:
one_layer_gnet_acc_train = []
one_layer_gnet_acc_test = []
time_per_image_train = []
time_per_image_test = []

data_dict = {'MODEL':[],
        'EPOCH':[],
        'TRAINACC':[],
        'TRAINLOSS':[],
        'TESTACC':[],
        'TESTLOSS':[],
        'TIME':[],
        'TRAINPRECISION':[],
        'TRAINRECALL':[],
        'TRAINF1SCORE':[],
        'TESTPRECISION':[],
        'TESTRECALL':[],
        'TESTF1SCORE':[],}

In [12]:
def setDataDict(model,epoch,trainacc,trainloss,testacc,testloss,time,trpre,trre,trf1,tepre,tere,tef1):
    data_dict['MODEL'].append(model)
    data_dict['EPOCH'].append(epoch)
    data_dict['TRAINACC'].append(trainacc)
    data_dict['TRAINLOSS'].append(trainloss)
    data_dict['TESTACC'].append(testacc)
    data_dict['TESTLOSS'].append(testloss)
    data_dict['TIME'].append(time)
    data_dict['TRAINPRECISION'].append(trpre)
    data_dict['TRAINRECALL'].append(trre)
    data_dict['TRAINF1SCORE'].append(trf1)
    data_dict['TESTPRECISION'].append(tepre)
    data_dict['TESTRECALL'].append(tere)
    data_dict['TESTF1SCORE'].append(tef1)

    

In [13]:
def trainingData(net,criterion,optimizer,epochs,modelName):
    for epoch in range(epochs):

        epochTime=time.perf_counter()
        running_loss = 0.0
        correct = 0
        trainrecall=[]
        trainprecision=[]
        testrecall=[]
        testprecision=[]
        label=[]
        net.train()
        start = time.perf_counter()
        for data in train:
            # get the inputs
            inputs, labels = data["image"], data["target"]

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs.to(device))
            loss = criterion(outputs, labels.to(device))
            loss.backward()
            optimizer.step()
            #scheduler.step()
            pred = outputs.max(1, keepdim=True)[1].to("cpu")

            correct += pred.eq(labels.view_as(pred)).sum().item()
            trainprecision.append(precision_score(labels.data, pred))
            trainrecall.append(recall_score(labels.data, pred))
            # print statistics
            running_loss += loss.item()
        finish = time.perf_counter()

        trainPrecisionScore=sum(trainprecision)/len(trainprecision)
        trainRecallScore=sum(trainrecall)/len(trainrecall)
        trainF1score=(2*(trainPrecisionScore*trainRecallScore))/(trainPrecisionScore+trainRecallScore)

        time_per_image_train.append((finish - start) / len(train_set))
        print(
            "[%d] train_acc: %.3f train_loss: %.3f"
            % (epoch + 1, correct / len(train_set), running_loss / len(train_set))
        )
        one_layer_gnet_acc_train.append(correct / len(train_set))
        train_loss=running_loss / len(train_set)
        train_acc=correct / len(train_set)
        running_loss = 0.0
        correct = 0
        start = time.perf_counter()
        with torch.no_grad():
            net.eval()
            for data in test:
                # get the inputs
                inputs, labels = data["image"], data["target"]

                # forward + backward + optimize
                outputs = net(inputs.to(device))
                loss = criterion(outputs, labels.to(device))

                pred = outputs.max(1, keepdim=True)[1].to("cpu")
                correct += pred.eq(labels.view_as(pred)).sum().item()
                testprecision.append(precision_score(labels.data, pred))
                testrecall.append(recall_score(labels.data, pred))
                running_loss += loss.item()
        finish = time.perf_counter()
        testPrecisionScore=sum(testprecision)/len(testprecision)
        testRecallScore=sum(testrecall)/len(testrecall)
        testF1score=(2*(testPrecisionScore*testRecallScore))/(testPrecisionScore+testRecallScore)
        time_per_image_test.append((finish - start) / len(test_set))
        print(
            "[%d] test_acc: %.3f test_loss: %.3f"
            % (epoch + 1, correct / len(test_set), running_loss / len(test_set))
        )
        one_layer_gnet_acc_test.append(correct / len(test_set))
        epochTime=time.perf_counter()-epochTime
        setDataDict(modelName,epoch+1,train_acc,train_loss,correct / len(test_set), running_loss / len(test_set),epochTime,trainPrecisionScore,trainRecallScore,trainF1score,testPrecisionScore,testRecallScore,testF1score)
        torch.save(net, modelName+"\\"+modelName+str(epoch+1)+".pt")
        print("Time Taken in Epoch "+ str(epoch+1)+"  Seconds: "+str(epochTime))

    print("Finished Training")

In [14]:
trainingData(net,criterion,optimizer,epochs,'GABORNN')

[1] train_acc: 0.505 train_loss: 0.005
[1] test_acc: 0.518 test_loss: 0.006
Time Taken in Epoch 1  Seconds: 1164.4910563
[2] train_acc: 0.538 train_loss: 0.005
[2] test_acc: 0.573 test_loss: 0.005
Time Taken in Epoch 2  Seconds: 1095.4786877000001
[3] train_acc: 0.563 train_loss: 0.005
[3] test_acc: 0.620 test_loss: 0.005
Time Taken in Epoch 3  Seconds: 1113.4954306
[4] train_acc: 0.592 train_loss: 0.005
[4] test_acc: 0.607 test_loss: 0.005
Time Taken in Epoch 4  Seconds: 1120.4467411000005
[5] train_acc: 0.619 train_loss: 0.005
[5] test_acc: 0.650 test_loss: 0.005
Time Taken in Epoch 5  Seconds: 1113.6744872
[6] train_acc: 0.630 train_loss: 0.005
[6] test_acc: 0.684 test_loss: 0.005
Time Taken in Epoch 6  Seconds: 1113.8237159
[7] train_acc: 0.645 train_loss: 0.005
[7] test_acc: 0.700 test_loss: 0.005
Time Taken in Epoch 7  Seconds: 1121.4635444999994
[8] train_acc: 0.662 train_loss: 0.005
[8] test_acc: 0.704 test_loss: 0.005
Time Taken in Epoch 8  Seconds: 1095.1785481999996
[9] trai

In [15]:
df=pd.read_csv("result.csv");

df=pd.concat([df,pd.DataFrame(data_dict)],ignore_index = True)
df.to_csv("result.csv", index=False)
data_dict = {'MODEL':[],
        'EPOCH':[],
        'TRAINACC':[],
        'TRAINLOSS':[],
        'TESTACC':[],
        'TESTLOSS':[],
        'TIME':[],
        'TRAINPRECISION':[],
        'TRAINRECALL':[],
        'TRAINF1SCORE':[],
        'TESTPRECISION':[],
        'TESTRECALL':[],
        'TESTF1SCORE':[],}

In [16]:
#torch.save(net.state_dict(), "gabornn.pt")
torch.save(net, "GABORNN.pt")

In [17]:
model = torch.load('GABORNN.pt')
model.eval()

GaborNN(
  (g1): GaborConv2dLayer(
    (conv_layer): Conv2d(3, 32, kernel_size=(15, 15), stride=(1, 1), bias=False)
  )
  (c1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2))
  (c2): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2))
  (fc1): Linear(in_features=6272, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=2, bias=True)
)

In [18]:
result_dict = {
    "train_acc": one_layer_gnet_acc_train[-1],
    "test_acc": one_layer_gnet_acc_test[-1],
    "time_per_image_train": sum(time_per_image_train) / len(time_per_image_train),
    "time_per_image_test": sum(time_per_image_test) / len(time_per_image_test),
}
with open("../metrics.json", "w+") as outfile:
    json.dump(result_dict, outfile)

In [19]:
epochs=30

In [20]:
from ModelNN import SimpleNN

In [21]:
net = SimpleNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters())
#scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.01, max_lr=0.1,cycle_momentum=False)

In [22]:
one_layer_gnet_acc_train = []
one_layer_gnet_acc_test = []
time_per_image_train = []
time_per_image_test = []

In [23]:
trainingData(net,criterion,optimizer,epochs,'SimpleNN')

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[1] train_acc: 0.528 train_loss: 0.005
[1] test_acc: 0.539 test_loss: 0.006
Time Taken in Epoch 1  Seconds: 1103.0474020000038
[2] train_acc: 0.558 train_loss: 0.005
[2] test_acc: 0.567 test_loss: 0.005
Time Taken in Epoch 2  Seconds: 1094.7783917999986
[3] train_acc: 0.575 train_loss: 0.005
[3] test_acc: 0.596 test_loss: 0.005
Time Taken in Epoch 3  Seconds: 1096.1629306000032
[4] train_acc: 0.591 train_loss: 0.005
[4] test_acc: 0.611 test_loss: 0.005
Time Taken in Epoch 4  Seconds: 1089.8836798000048
[5] train_acc: 0.593 train_loss: 0.005
[5] test_acc: 0.603 test_loss: 0.005
Time Taken in Epoch 5  Seconds: 1087.9936659000014
[6] train_acc: 0.593 train_loss: 0.005
[6] test_acc: 0.618 test_loss: 0.005
Time Taken in Epoch 6  Seconds: 1080.6562752999962
[7] train_acc: 0.611 train_loss: 0.005
[7] test_acc: 0.630 test_loss: 0.005
Time Taken in Epoch 7  Seconds: 1080.0842963999967
[8] train_acc: 0.623 train_loss: 0.005
[8] test_acc: 0.643 test_loss: 0.005
Time Taken in Epoch 8  Seconds: 108

In [24]:

torch.save(net, "SimpleNN.pt")

In [25]:
model = torch.load('SimpleNN.pt')
model.eval()

SimpleNN(
  (c1): Conv2d(3, 32, kernel_size=(15, 15), stride=(1, 1))
  (c2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2))
  (c3): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2))
  (fc1): Linear(in_features=6272, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=2, bias=True)
)

In [26]:
df=pd.read_csv("result.csv");

df=pd.concat([df,pd.DataFrame(data_dict)],ignore_index = True)
df.to_csv("result.csv", index=False)
data_dict = {'MODEL':[],
        'EPOCH':[],
        'TRAINACC':[],
        'TRAINLOSS':[],
        'TESTACC':[],
        'TESTLOSS':[],
        'TIME':[],
        'TRAINPRECISION':[],
        'TRAINRECALL':[],
        'TRAINF1SCORE':[],
        'TESTPRECISION':[],
        'TESTRECALL':[],
        'TESTF1SCORE':[],}

In [34]:
epochs=30

In [35]:
from GaborNN5layer import GaborNN5layer

In [36]:
net = GaborNN5layer().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters())
#scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.01, max_lr=0.1,cycle_momentum=False)

In [37]:
one_layer_gnet_acc_train = []
one_layer_gnet_acc_test = []
time_per_image_train = []
time_per_image_test = []
epochs=30

In [38]:
trainingData(net,criterion,optimizer,epochs,'GaborNN5layer')

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision 

[1] train_acc: 0.497 train_loss: 0.005
[1] test_acc: 0.518 test_loss: 0.006
Time Taken in Epoch 1  Seconds: 1143.6660904000018
[2] train_acc: 0.523 train_loss: 0.005
[2] test_acc: 0.553 test_loss: 0.005
Time Taken in Epoch 2  Seconds: 1139.3039456000115
[3] train_acc: 0.539 train_loss: 0.005
[3] test_acc: 0.542 test_loss: 0.005
Time Taken in Epoch 3  Seconds: 1151.366920200002


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[4] train_acc: 0.537 train_loss: 0.005
[4] test_acc: 0.546 test_loss: 0.005
Time Taken in Epoch 4  Seconds: 1137.5177123999893
[5] train_acc: 0.586 train_loss: 0.005
[5] test_acc: 0.627 test_loss: 0.005
Time Taken in Epoch 5  Seconds: 1137.7180198999995
[6] train_acc: 0.624 train_loss: 0.005
[6] test_acc: 0.684 test_loss: 0.005
Time Taken in Epoch 6  Seconds: 1136.7451956999867
[7] train_acc: 0.647 train_loss: 0.005
[7] test_acc: 0.686 test_loss: 0.005
Time Taken in Epoch 7  Seconds: 1138.260922699992
[8] train_acc: 0.668 train_loss: 0.005
[8] test_acc: 0.706 test_loss: 0.004
Time Taken in Epoch 8  Seconds: 1135.7440130999894
[9] train_acc: 0.674 train_loss: 0.005
[9] test_acc: 0.690 test_loss: 0.005
Time Taken in Epoch 9  Seconds: 1138.4697948000103
[10] train_acc: 0.684 train_loss: 0.005
[10] test_acc: 0.730 test_loss: 0.004
Time Taken in Epoch 10  Seconds: 1144.1515679000004
[11] train_acc: 0.688 train_loss: 0.004
[11] test_acc: 0.726 test_loss: 0.004
Time Taken in Epoch 11  Seconds

In [39]:
torch.save(net, "GaborNN5layer.pt")

In [40]:
df=pd.read_csv("result.csv");

df=pd.concat([df,pd.DataFrame(data_dict)],ignore_index = True)
df.to_csv("result.csv", index=False)
data_dict = {'MODEL':[],
        'EPOCH':[],
        'TRAINACC':[],
        'TRAINLOSS':[],
        'TESTACC':[],
        'TESTLOSS':[],
        'TIME':[],
        'TRAINPRECISION':[],
        'TRAINRECALL':[],
        'TRAINF1SCORE':[],
        'TESTPRECISION':[],
        'TESTRECALL':[],
        'TESTF1SCORE':[],}

In [41]:
model = torch.load('GaborNN5layer.pt')
model.eval()

GaborNN5layer(
  (g1): GaborConv2dLayer(
    (conv_layer): Conv2d(3, 32, kernel_size=(15, 15), stride=(1, 1), bias=False)
  )
  (c1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2))
  (c2): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2))
  (c3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1))
  (c4): Conv2d(256, 512, kernel_size=(1, 1), stride=(1, 1))
  (fc1): Linear(in_features=25088, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=2, bias=True)
)

In [42]:
from GaborNNWithBN import GaborNNWithBN

In [43]:
net = GaborNNWithBN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters())
#scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.01, max_lr=0.1,cycle_momentum=False)

In [44]:
one_layer_gnet_acc_train = []
one_layer_gnet_acc_test = []
time_per_image_train = []
time_per_image_test = []
epochs=30

In [45]:
trainingData(net,criterion,optimizer,epochs,'GaborNNWithBN')

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[1] train_acc: 0.536 train_loss: 0.006
[1] test_acc: 0.586 test_loss: 0.005
Time Taken in Epoch 1  Seconds: 1183.140026499983
[2] train_acc: 0.587 train_loss: 0.005
[2] test_acc: 0.628 test_loss: 0.005
Time Taken in Epoch 2  Seconds: 1160.1024382000032
[3] train_acc: 0.611 train_loss: 0.005
[3] test_acc: 0.651 test_loss: 0.005
Time Taken in Epoch 3  Seconds: 1089.3362362000044
[4] train_acc: 0.627 train_loss: 0.005
[4] test_acc: 0.657 test_loss: 0.005
Time Taken in Epoch 4  Seconds: 1091.065654200007
[5] train_acc: 0.638 train_loss: 0.005
[5] test_acc: 0.677 test_loss: 0.005
Time Taken in Epoch 5  Seconds: 1124.5300399999833
[6] train_acc: 0.653 train_loss: 0.005
[6] test_acc: 0.678 test_loss: 0.005
Time Taken in Epoch 6  Seconds: 1126.738545100001
[7] train_acc: 0.662 train_loss: 0.005
[7] test_acc: 0.697 test_loss: 0.005
Time Taken in Epoch 7  Seconds: 1136.7381032999838
[8] train_acc: 0.673 train_loss: 0.005
[8] test_acc: 0.723 test_loss: 0.004
Time Taken in Epoch 8  Seconds: 1165.9

In [46]:
torch.save(net, "GaborNNWithBN.pt")

In [47]:
df=pd.read_csv("result.csv");

df=pd.concat([df,pd.DataFrame(data_dict)],ignore_index = True)
df.to_csv("result.csv", index=False)
data_dict = {'MODEL':[],
        'EPOCH':[],
        'TRAINACC':[],
        'TRAINLOSS':[],
        'TESTACC':[],
        'TESTLOSS':[],
        'TIME':[],
        'TRAINPRECISION':[],
        'TRAINRECALL':[],
        'TRAINF1SCORE':[],
        'TESTPRECISION':[],
        'TESTRECALL':[],
        'TESTF1SCORE':[],}

In [48]:
model = torch.load('GaborNNWithBN.pt')
model.eval()

GaborNNWithBN(
  (g1): GaborConv2dLayer(
    (conv_layer): Conv2d(3, 32, kernel_size=(15, 15), stride=(1, 1), bias=False)
  )
  (c1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (c2): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2))
  (c3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1))
  (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (c4): Conv2d(256, 512, kernel_size=(1, 1), stride=(1, 1))
  (fc1): Linear(in_features=25088, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=2, bias=True)
)

In [49]:
net =torch.load('GABORNN.pt')
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters())
#scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.01, max_lr=0.1,cycle_momentum=False)

In [50]:
data_dict = {'MODEL':[],
        'EPOCH':[],
        'TRAINACC':[],
        'TRAINLOSS':[],
        'TESTACC':[],
        'TESTLOSS':[],
        'TIME':[],
        'TRAINPRECISION':[],
        'TRAINRECALL':[],
        'TRAINF1SCORE':[],
        'TESTPRECISION':[],
        'TESTRECALL':[],
        'TESTF1SCORE':[],}

In [51]:
one_layer_gnet_acc_train = []
one_layer_gnet_acc_test = []
time_per_image_train = []
time_per_image_test = []
epochs=30

In [53]:
from GaborPretrainedModel import GaborPretrainedModel

In [54]:
net =GaborPretrainedModel.VGGNET16().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters())

In [55]:
one_layer_gnet_acc_train = []
one_layer_gnet_acc_test = []
time_per_image_train = []
time_per_image_test = []
epochs=30

In [56]:
trainingData(net,criterion,optimizer,epochs,'VGGNET16')

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[1] train_acc: 0.568 train_loss: 0.005
[1] test_acc: 0.630 test_loss: 0.005
Time Taken in Epoch 1  Seconds: 9854.303953500028
[2] train_acc: 0.586 train_loss: 0.005
[2] test_acc: 0.587 test_loss: 0.005
Time Taken in Epoch 2  Seconds: 9863.037573199981
[3] train_acc: 0.589 train_loss: 0.005
[3] test_acc: 0.600 test_loss: 0.005
Time Taken in Epoch 3  Seconds: 9854.544596799999
[4] train_acc: 0.592 train_loss: 0.005
[4] test_acc: 0.619 test_loss: 0.005
Time Taken in Epoch 4  Seconds: 9775.889253100002
[5] train_acc: 0.600 train_loss: 0.005
[5] test_acc: 0.625 test_loss: 0.005
Time Taken in Epoch 5  Seconds: 14488.72169939999
[6] train_acc: 0.604 train_loss: 0.005
[6] test_acc: 0.661 test_loss: 0.005
Time Taken in Epoch 6  Seconds: 12614.209830000007
[7] train_acc: 0.595 train_loss: 0.005
[7] test_acc: 0.636 test_loss: 0.005
Time Taken in Epoch 7  Seconds: 12731.281141499989
[8] train_acc: 0.596 train_loss: 0.005
[8] test_acc: 0.625 test_loss: 0.005
Time Taken in Epoch 8  Seconds: 12491.81

In [57]:
torch.save(net, "VGGNET16.pt")

In [58]:
df=pd.read_csv("result.csv");

df=pd.concat([df,pd.DataFrame(data_dict)],ignore_index = True)
df.to_csv("result.csv", index=False)
data_dict = {'MODEL':[],
        'EPOCH':[],
        'TRAINACC':[],
        'TRAINLOSS':[],
        'TESTACC':[],
        'TESTLOSS':[],
        'TIME':[],
        'TRAINPRECISION':[],
        'TRAINRECALL':[],
        'TRAINF1SCORE':[],
        'TESTPRECISION':[],
        'TESTRECALL':[],
        'TESTF1SCORE':[],}

In [59]:
from GaborNNWithResnet import GaborNNWithResnet

In [60]:
net =GaborNNWithResnet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters())

In [61]:
one_layer_gnet_acc_train = []
one_layer_gnet_acc_test = []
time_per_image_train = []
time_per_image_test = []
epochs=30

In [62]:
trainingData(net,criterion,optimizer,epochs,'RESNET18')

[1] train_acc: 0.633 train_loss: 0.005
[1] test_acc: 0.680 test_loss: 0.005


FileNotFoundError: [Errno 2] No such file or directory: 'RESNET18\\RESNET181.pt'

In [ ]:
torch.save(net, "RESNET18.pt")

In [ ]:
df=pd.read_csv("result.csv");

df=pd.concat([df,pd.DataFrame(data_dict)],ignore_index = True)
df.to_csv("result.csv", index=False)
data_dict = {'MODEL':[],
        'EPOCH':[],
        'TRAINACC':[],
        'TRAINLOSS':[],
        'TESTACC':[],
        'TESTLOSS':[],
        'TIME':[],
        'TRAINPRECISION':[],
        'TRAINRECALL':[],
        'TRAINF1SCORE':[],
        'TESTPRECISION':[],
        'TESTRECALL':[],
        'TESTF1SCORE':[],}